In [1]:
# modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import chardet
import csv
import os
import matplotlib.dates as mdates
from IPython.display import Image
from scipy import stats

## Inserez les chemins vers le fichiers CSV:

In [2]:
# on récupère les informations du dossier
dossier = './raw_data'
contenuDossier = os.listdir(dossier)
print(contenuDossier)
nombrePoints = 0
nomPoints = []
numeroPoints = []
# on parcours le dossier en recherchant les fichiers des points
for x in contenuDossier:
    if x.startswith('point') :
        nombrePoints += 1
        nomPoints.append(x)
        numeroPoints.append(x[5:7])
print(nomPoints)
# nomPoints.remove('point51_131023')
# nomPoints.remove('point50_131023')
numeroPoints.remove('15')
# numeroPoints.remove('50')
# numeroPoints.remove('51')
print(numeroPoints)

['.DS_Store', 'Point034', 'point13_29_07_15', 'point14_29_07_15', 'point15_29_07_15', 'schema_dispositif_AvAv_23_07_15.png']
['point13_29_07_15', 'point14_29_07_15', 'point15_29_07_15']
['13', '14']


In [3]:
def read_csv (chemin_fichier):
    #Detecter separateur
    with open(chemin_fichier, 'r') as file:
        sniffer = csv.Sniffer()
        sample_data = file.read(1024)
        detecter_separateur = (sniffer.sniff(sample_data).delimiter)

    if "Titre" in open(chemin_fichier).readline():
        data_frame = pd.read_csv(chemin_fichier, sep=detecter_separateur, skiprows=1)
    else:
        data_frame = pd.read_csv(chemin_fichier, sep=detecter_separateur)

    return data_frame 

In [4]:
# on crée un dictionnaire avec toutes les informations pour chaque point et on met les dictionnaires dans une liste
data = []
for x in nomPoints :
    dico = {}
    # le numéro
    dico['numero'] = x[5:7]
    # le nom
    dico['nom'] = x
    # la date
    dico['date'] = x[8:18]
    # le chemin vers le dossier
    dico['chemin'] = dossier + '/' + x
    with open(dico['chemin'] + '/geometrie.txt', 'r') as fichier:
        lignes = fichier.readlines()
        # le nom du capteur
        dico['capteur'] = lignes[1][:-1]
        # la profondeur du capteur
        if lignes[3] == '\n' :
            dico['profondeur'] = 0
        else : 
            dico['profondeur'] = lignes[3]
        # les profondeurs auquelles on fait les mesures
        dico['profondeurMesures'] = []
        profMes = lignes[5].split(';')
        for num in profMes :
            dico['profondeurMesures'].append(int(num))
    # on va ensuite mettre les dataframes dans le dictionnaire
    contenuDossierPoint = os.listdir(dico['chemin'])
    for x in contenuDossierPoint:
        # la pression
        if (x.startswith('p') and x.endswith('.csv')) and not x.endswith('cleaned.csv') :
            dico['pression'] = read_csv(dico['chemin'] + '/' + x)
        # la température
        if x.startswith('t') and x.endswith('.csv'):
            dico['temperature'] = read_csv(dico['chemin'] + '/' + x)
    # dans les configurations du capteur on va chercher les données
    chemin_etalonnage = 'configuration/pressure_sensors/P' + dico['capteur'][1:] +'.csv'
    if os.path.exists(chemin_etalonnage):
        with open(chemin_etalonnage) as fichier :
            lignes = fichier.readlines()
            for l in lignes :
                if l.startswith('Intercept') :
                    dico['intercept'] = float(l[10:-2])
                if l.startswith('dU/dH') :
                    dico['dU/dH'] = float(l[6:-2])
                if l.startswith('dU/dT') :
                    dico['dU/dT'] = float(l[6:-2])
    data.append(dico)


In [5]:


data[6]['pression']['Date Heure, GMT+01:00'].is_monotonic_increasing
#data[5]['numero']

IndexError: list index out of range

In [ ]:
# en faisant data[0] on a accès à toutes les informations du point 1
data[2]

{'numero': '15',
 'nom': 'point15_29_07_15',
 'date': '29_07_15',
 'chemin': './raw_data/point15_29_07_15',
 'capteur': 'p506',
 'profondeur': 0,
 'profondeurMesures': [7, 22, 37, 52],
 'temperature':        # Date Heure, GMT+02:00  \
 0      1  07/23/15 02:00:00 PM   
 1      2  07/23/15 02:15:00 PM   
 2      3  07/23/15 02:30:00 PM   
 3      4  07/23/15 02:45:00 PM   
 4      5  07/23/15 03:00:00 PM   
 ..   ...                   ...   
 559  560  07/29/15 09:45:00 AM   
 560  561  07/29/15 10:00:00 AM   
 561  562  07/29/15 10:15:00 AM   
 562  563  07/29/15 10:30:00 AM   
 563  564  07/29/15 10:45:00 AM   
 
      Temp. °C (LGR S/N: 10445474, SEN S/N: 10445474)  \
 0                                             28.320   
 1                                             27.333   
 2                                             27.505   
 3                                             27.949   
 4                                             26.671   
 ..                                 

# Est ce que les fichiers peuvent être utilisés ?

**Dans notre cas, nous prendrons ces solutions**

* S'il manque toutes les données du capteurs de pression ou l'étalonnage, nous déciderons que ce point est invalide.
* S'il manque une partie des données nous vérifierons que nous avons au mimimun 7 jours de données
* S'il manque plus de 2 capteurs de température, nous déciderons que ce point est invalide.

### Est ce que nous avons les données de l'étalonnage du capteur ?

In [ ]:
for x in data :
    if not 'intercept' in x.keys() :
        print('le point '+ x['numero'] + ' n\'a pas d\'étalonnage, il n\'est pas valide')
        data.remove(x)

### Est ce que nous avons des données pour le capteur de pression ?

In [ ]:
for x in data :
    if x['pression'].empty :
        print('le point '+ x['numero'] + ' n\'a pas de données de pression, il n\'est pas valide')
        data.remove(x)

In [ ]:
data[5]

{'numero': '13',
 'nom': 'point13_29_07_15',
 'date': '29_07_15',
 'chemin': './raw_data/point13_29_07_15',
 'capteur': 'p505',
 'profondeur': 0,
 'profondeurMesures': [10, 20, 30, 40],
 'pression':        # Date Heure, GMT+02:00  \
 0      1  07/23/15 02:00:00 PM   
 1      2  07/23/15 02:15:00 PM   
 2      3  07/23/15 02:30:00 PM   
 3      4  07/23/15 02:45:00 PM   
 4      5  07/23/15 03:00:00 PM   
 ..   ...                   ...   
 558  559  07/29/15 09:30:00 AM   
 559  560  07/29/15 09:45:00 AM   
 560  561  07/29/15 10:00:00 AM   
 561  562  07/29/15 10:15:00 AM   
 562  563  07/29/15 10:30:00 AM   
 
      Tension, V (LGR S/N: 10117169, SEN S/N: 10117169)  \
 0                                              0.58181   
 1                                              0.58059   
 2                                              0.57937   
 3                                              0.57937   
 4                                              0.57937   
 ..                       

### On modifie les nom des colonnes du dataframe

In [ ]:
# pour les températures
l=0
for x in data :
    # on supprime les colonnes qui ne servent à rien
    for i in x['temperature'].columns :
        if not( i.startswith(('Date', 'dates', 'temp','Temp'))) :
            del x['temperature'][i]
    for i in x['pression'].columns :
        if not( i.startswith(('Date', 'dates', 'Tension', 'tension', 'pressure', 'temp', 'Temp'))) :
            del x['pression'][i]
    # on renomme les colonnes
    profondeur = x['profondeurMesures']
    #print(x['profondeurMesures'])
    colonnesT = ['dates']
    for num in profondeur :
        colonnesT.append('Temp_profondeur_'+ str(num))
    #print(colonnesT)
    # print(x['temperature'].columns)
    x['temperature'].columns = colonnesT
    colonnesP = ['dates', 'tension', 'temperature_stream']
    x['pression'].columns = colonnesP
    l+=1
    

In [ ]:
# notamment la température et la pression
data[6]['temperature'].head(5)

,dates,Temp_profondeur_10,Temp_profondeur_20,Temp_profondeur_30,Temp_profondeur_40
0,07/12/16 02:00:00 PM,23.785,25.234,24.460,24.412
1,07/12/16 02:15:00 PM,17.463,16.892,16.677,537.327
2,07/12/16 02:30:00 PM,16.749,16.320,16.129,537.327
3,07/12/16 02:45:00 PM,16.725,16.272,16.082,537.327
4,07/12/16 03:00:00 PM,16.749,16.272,16.058,537.327


## Conversion de valeurs de tension a differance de charge

In [ ]:
for point in data:
    k0 = float(point['intercept'])
    k1 = float(point['dU/dH'])
    k2 = float(point['dU/dT'])
    
    # Calcul de la colonne 'dH' en utilisant la formule H = 1/k1 * (U - k0 - k2 * T)
    U = point['pression']['tension'].astype(float)
    T = point['pression']['temperature_stream'].astype(float)
    point['pression']['dH'] = (1 / k1) * (U - k0 - k2 * T)

data[0]['pression']

,dates,tension,temperature_stream,dH
0,07/12/16 12:00:00 PM,1.56471,20.007,-0.222196
1,07/12/16 12:15:00 PM,0.62210,14.361,0.416653
2,07/12/16 12:30:00 PM,0.63614,14.314,0.407006
3,07/12/16 12:45:00 PM,0.69780,14.218,0.364748
4,07/12/16 01:00:00 PM,0.66911,14.098,0.384245
...,...,...,...,...
2106,08/03/16 10:30:00 AM,2.49939,14.314,-0.867020
2107,08/03/16 10:45:00 AM,2.49939,17.558,-0.863760
2108,08/03/16 11:00:00 AM,2.49939,18.129,-0.863186
2109,08/03/16 11:06:59 AM,NaN,NaN,NaN


### On enlève les valeurs Nan

In [ ]:
for x in data :
    x['temperature'] = x['temperature'].dropna(axis = 0, how = 'any')
    x['pression'] = x['pression'].dropna(axis = 0, how='any')

In [ ]:
from dateutil import parser

# Conversion dans le bon format date
for x in data:
    x['pression']['dates'] = x['pression']['dates'].apply(lambda x: parser.parse(x).strftime('%d-%m-%Y %H:%M'))
    x['temperature']['dates'] = x['temperature']['dates'].apply(lambda x: parser.parse(x).strftime('%d-%m-%Y %H:%M'))
#    point['pression']['dates'] = pd.to_datetime(point['pression']['dates'], format='mixed')
#    point['temperature']['dates'] = pd.to_datetime(point['temperature']['dates'], format='mixed')

/var/folders/95/14nx391x279gt37l3hnx_p_80000gn/T/ipykernel_87913/1590845049.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['pression']['dates'] = x['pression']['dates'].apply(lambda x: parser.parse(x).strftime('%d-%m-%Y %H:%M'))


In [ ]:
data[0]['pression'].head(5)

,dates,tension,temperature_stream,dH
0,12-07-2016 12:00,1.56471,20.007,-0.222196
1,12-07-2016 12:15,0.62210,14.361,0.416653
2,12-07-2016 12:30,0.63614,14.314,0.407006
3,12-07-2016 12:45,0.69780,14.218,0.364748
4,12-07-2016 13:00,0.66911,14.098,0.384245


In [ ]:
for x in data :
    # Traiter chaque colonne sauf la colonne des dates
    columns_to_processP = [col for col in x['pression'].columns if col != 'dates']
    columns_to_processT = [col for col in x['temperature'].columns if col != 'dates']
    # Créer un DataFrame vide pour stocker les données traitées
    x['pression2'] = x['pression'][['dates']].copy()
    x['temperature2'] = x['temperature'][['dates']].copy()
    # Boucler à travers chaque colonne à traiter
    for column_name in columns_to_processP :
        # Calculer le Z-score pour la colonne
        z_scores = np.abs(stats.zscore(x['pression'][column_name]))
        # Définir un seuil pour le Z-score (par exemple, 3)
        threshold = 3
        # Sélectionner les lignes avec des Z-scores inférieurs au seuil
        df_cleaned = x['pression'][z_scores < threshold] 
        # Copier les données traitées dans df_processed
        x['pression2'][column_name] = df_cleaned[column_name]
    # pareil pour les températures
    for column_name in columns_to_processT :
        # Calculer le Z-score pour la colonne
        z_scores = np.abs(stats.zscore(x['temperature'][column_name]))
        # Définir un seuil pour le Z-score (par exemple, 3)
        threshold = 3
        # Sélectionner les lignes avec des Z-scores inférieurs au seuil
        df_cleaned = x['temperature'][z_scores < threshold] 
        # Copier les données traitées dans df_processed
        x['temperature2'][column_name] = df_cleaned[column_name]

In [ ]:
def nb_val_abberantes(capteur_riviere, capteur_ZH):
    df = capteur_ZH.copy()
    df["temperature_stream"] = capteur_riviere[["temperature_stream"]]
    df["dH"] = capteur_riviere[["dH"]]
    for column in df.columns:
        if column != "dates":
            print(f"Analyse pour la colonne : {column}")
            #plot_data_with_and_without_outliers(df, column)
            #print(f"Nombre de valeurs aberrantes : {count_outliers(df, column)}\n")
            
    # Créer un DataFrame vide pour stocker les données traitées
    df_processed = df[["dates"]].copy()

    # Boucler à travers chaque colonne de df
    for column_name in df.columns:
        if column_name == "dates":
            continue  # Ignorer la colonne des dates
            
        # Traiter la colonne et stocker les données traitées dans df_processed
        median, q1, q3, iqr, minimum, maximum = calculate_quantiles_iqr(df, column_name)
        df_processed[f"{column_name}"] = remove_outliers(df, column_name)[column_name]
        
    # Afficher les statistiques des données traitées
    print(df_processed.describe())
    return df, df_processed

In [ ]:
# Fonction pour calculer les statistiques et les valeurs IQR
def calculate_quantiles_iqr(df, column_name):
    median = df[column_name].median()
    # Calculer les quantiles Q1 et Q3 pour la colonne spécifiée
    q1 = df[column_name].quantile(0.25)
    q3 = df[column_name].quantile(0.75)
    
    # Calculer l'IQR
    iqr = q3 - q1
    minimum = q1 - 1.5 * iqr  # Valeur minimale (|- dans la boîte à moustaches)
    maximum = q3 + 1.5 * iqr  # Valeur maximale (-| dans la boîte à moustaches)
    return median, q1, q3, iqr, minimum, maximum

# Fonction pour obtenir un texte décrivant les statistiques de base
def get_iqr_text(df, col_name):
    # Calcul des statistiques de base
    median, q1, q3, iqr, minimum, maximum = calculate_quantiles_iqr(df, col_name)
    
    # Création du texte
    text = f"médiane={median:.2f}, Q1={q1:.2f}, Q3={q3:.2f}, IQR={iqr:.2f}, minimum={minimum:.2f}, maximum={maximum:.2f}"
    return text

# Fonction pour supprimer les valeurs aberrantes
def remove_outliers(df, col_name):
    # Obtention des limites pour les valeurs non aberrantes
    _, _, _, _, minimum, maximum = calculate_quantiles_iqr(df, col_name)
    
    # Sélection des lignes avec des valeurs non aberrantes
    df_out = df.loc[(df[col_name] > minimum) & (df[col_name] < maximum)]
    return df_out

# Fonction pour compter les valeurs aberrantes
def count_outliers(df_in, col_name):
    # Obtention des limites pour les valeurs aberrantes
    _, _, _, _, minimum, maximum = calculate_quantiles_iqr(df_in, col_name)
    
    # Sélection des lignes avec des valeurs aberrantes
    df_outliers = df_in.loc[(df_in[col_name] <= minimum) | (df_in[col_name] >= maximum)]
    
    # Comptage des valeurs aberrantes
    num_outliers = df_outliers.shape[0]
    return num_outliers

# Fonction pour générer un graphique de boîte à moustaches
def box_and_whisker(df_in, col_name):
    # Obtention du texte décrivant les statistiques
    title = get_iqr_text(df_in, col_name)
    
    # Tracé du graphique de boîte à moustaches
    sns.boxplot(df_in[col_name])
    plt.title(title)
    plt.show()

# Fonction pour tracer les données avec et sans valeurs aberrantes
def plot_data_with_and_without_outliers(df, col_name):
    plt.figure(figsize=(12, 6))
    
    # Tracé des données brutes
    plt.subplot(1, 2, 1)
    plt.title("Données brutes")
    sns.boxplot(df[col_name])
    
    # Tracé des données sans valeurs aberrantes
    plt.subplot(1, 2, 2)
    plt.title("Données sans valeurs aberrantes")
    df_cleaned = remove_outliers(df, col_name)
    sns.boxplot(df_cleaned[col_name])
    
    plt.tight_layout()
    plt.show()


In [ ]:
for x in data :
    df_processed  = nb_val_abberantes(x['pression'], x['temperature'])[1]
    df_processed = df_processed.dropna(axis = 0, how='any')
    x['pression2'] = df_processed[['dates', 'temperature_stream', 'dH']].copy()
    x['temperature2'] = df_processed.drop(columns = [ 'temperature_stream','dH']).copy()

Analyse pour la colonne : Temp_profondeur_10
Analyse pour la colonne : Temp_profondeur_20
Analyse pour la colonne : Temp_profondeur_30
Analyse pour la colonne : Temp_profondeur_40
Analyse pour la colonne : temperature_stream
Analyse pour la colonne : dH
       Temp_profondeur_10  Temp_profondeur_20  Temp_profondeur_30  \
count         2090.000000         2044.000000         2024.000000   
mean            14.910749           14.063272           13.789086   
std              0.679988            0.366243            0.254697   
min             13.401000           13.137000           13.161000   
25%             14.433000           13.858000           13.714000   
50%             14.888000           14.098000           13.858000   
75%             15.366000           14.337000           13.978000   
max             16.820000           14.888000           14.385000   

       Temp_profondeur_40  temperature_stream           dH  
count         2045.000000         2106.000000  1820.000000  
me

In [ ]:
for x in data :
    x['temperature2'] = x['temperature2'].dropna(axis = 0, how = 'any')
    x['pression2'] = x['pression2'].dropna(axis = 0, how='any')

In [ ]:
# on met à jour la date de début de la mesure, la date de fin et la période
from datetime import datetime

for x in data :
    x['dateDebut'] = datetime.strptime(x['pression2']['dates'].iloc[0], '%d-%m-%Y %H:%M')
    x['dateFin'] = datetime.strptime(x['pression2']['dates'].iloc[x['pression2']['dates'].shape[0]-1], '%d-%m-%Y %H:%M')
    x['periode'] = x['dateFin'] - x['dateDebut']

In [ ]:
# for x in data :
#     x['pression2']['dates'].apply(lambda x: datetime.strptime(x, '%d-%m-%Y %H:%M'))
#     x['temperature2']['dates'].apply(lambda x: datetime.strptime(x, '%d-%m-%Y %H:%M'))
#     x['pression']['dates'].apply(lambda x: datetime.strptime(x, '%d-%m-%Y %H:%M'))
#     x['temperature']['dates'].apply(lambda x: datetime.strptime(x, '%d-%m-%Y %H:%M'))

### Est ce que nous avons au minimum 3 jours de données ?

In [ ]:
for x in data :
    if x['periode'].days <= 3 :
        print('le point ' + x['numero'] + ' n\'est pas valide car sa période est de seulement ' + str(x['periode'])[0] +' jours')
        data.remove(x)

le point 15 n'est pas valide car sa période est de seulement 0 jours


### Création du fichier `info.txt`

In [ ]:
# Ouvrez un fichier en mode écriture ('w')
for x in data :
    if os.path.exists(x['chemin'] + '/point' + x['numero'] + '_info.txt') :
        os.remove(x['chemin'] + '/point' + x['numero'] + '_info.txt')
    with open(x['chemin'] + '/point' + x['numero'] + '_info.txt', 'w') as fichier:
        # Parcourez le dictionnaire et écrivez les données dans le fichier
        fichier.write('Point_Name,Point' + x['numero'] + '\n')
        fichier.write('P_Sensor_Name,' + x['capteur'] + '\n')
        fichier.write('Implantation_Date,' + x['date'] + '\n')
        fichier.write('Meas_date,' + str(x['dateDebut']) + '\n')
        fichier.write('River_Deb,' '\n') 
        fichier.write('Delta_h,' + '\n')
        fichier.write('Periode,' + str(x['periode']) + '\n')

In [ ]:
data[2]['temperature2']

,dates,Temp_profondeur_10,Temp_profondeur_20,Temp_profondeur_30,Temp_profondeur_40
198,15-10-2023 11:15,12.251,13.102,13.617,14.018
199,15-10-2023 11:30,12.251,13.094,13.601,14.005
200,15-10-2023 11:45,12.263,13.087,13.586,13.990
201,15-10-2023 12:00,12.252,13.079,13.571,13.976
202,15-10-2023 12:15,12.245,13.072,13.557,13.963
...,...,...,...,...,...
1713,31-10-2023 06:00,12.567,12.803,12.906,13.020
1714,31-10-2023 06:15,12.554,12.793,12.901,13.019
1715,31-10-2023 06:30,12.544,12.784,12.895,13.016
1716,31-10-2023 06:45,12.533,12.773,12.889,13.014


# Traitement des signaux de temperature et pression

### Représentation de la corrélation temperature pression (original) Point13

In [ ]:
TsC = data[5]['pression2']['temperature_stream']
dH = data[5]['pression2']['dH']
Dates = data[5]['pression2']['dates']

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.set_ylabel("y_T")
ax1.set_ylim(13, 19)
ax2.set_ylabel("y_dH")
ax2.set_ylim(0.084, 0.096)
ax1.plot(Dates, TsC, color="red", label="TsC_original_p13")
ax2.plot(Dates, dH, color="blue", label="dH_original_p13")
ax1.set_title("TsC_dH_p13")

fig, ax = plt.subplots(figsize=(5,5))
ax.scatter(TsC, dH, color="red", s=5)
ax.set_xlabel("TsC_p13")
ax.set_ylabel("dH_p13")
ax.set_title("TsC_dH_p13")

plt.tight_layout()
plt.show()

### Filtre passe-bas

In [ ]:
# Butterworth filter method

from scipy import signal

data_to_filter = dH
order = 2
cutoff_freq = 2*(15/60/30)

b, a = signal.butter(order, cutoff_freq, 'lowpass')

filtered_data_low = signal.lfilter(b, a, data_to_filter)

In [ ]:
filtered_dH_low = pd.DataFrame({'filtered_dH': filtered_data_low, 'dates': Dates})

fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(Dates, dH, color="red", label="dH_original_p13", s=5)
ax.plot(filtered_dH_low["dates"], filtered_dH_low["filtered_dH"], color="blue", label="filtered_dH_p13")
ax.set_xlabel("Date")
ax.set_ylabel("dH")
ax.set_title("Time Series Comparison ")
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Butterworth two-way-filter method

forward_filtered_signal_low = signal.lfilter(b, a, data_to_filter)

reversed_signal = data_to_filter[::-1]
backward_filtered_signal = signal.lfilter(b, a, reversed_signal)
backward_filtered_signal_low = backward_filtered_signal[::-1]

In [ ]:
filtered_dH_low_forward = pd.DataFrame({'filtered_dH': forward_filtered_signal_low, 'dates': Dates})
filtered_dH_low_backward = pd.DataFrame({'filtered_dH': backward_filtered_signal_low, 'dates': Dates})

fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(Dates, dH, color="red", label="dH_original_p13", s=5)
ax.plot(filtered_dH_low_forward["dates"], filtered_dH_low_forward["filtered_dH"], color="blue", label="filtered_dH_forward_p13")
ax.set_xlabel("Date")
ax.set_ylabel("dH_forward")
ax.set_title("Time Series Comparison" )
ax.legend()

fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(Dates, dH, color="red", label="dH_original_p13", s=5)
ax.plot(filtered_dH_low_backward["dates"], filtered_dH_low_backward["filtered_dH"], color="blue", label="filtered_dH_backward_p13")
ax.set_xlabel("Date")
ax.set_ylabel("dH_backward")
ax.set_title("Time Series Comparison")
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Combination of two-way-filtered signal with a weight fonction

L = len(data_to_filter)
weight = np.zeros(L)
combined_filtered_signal_low = np.zeros(L)

a = 1.0 / L
for i in range(L):
    weight[i] = i * a

for i in range(L):
    combined_filtered_signal_low[i] = forward_filtered_signal_low[i] * weight[i] + backward_filtered_signal_low[i] * (1-weight[i])

In [ ]:
combined_filtered_dH_low_p13 = pd.DataFrame({'combined_filtered_dH': combined_filtered_signal_low, 'dates': Dates})

fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(Dates, dH, color="red", label="dH_original_p13", s=5)
ax.plot(combined_filtered_dH_low_p13["dates"], combined_filtered_dH_low_p13["combined_filtered_dH"], color="blue", label="combined_filtered_dH_p13")
ax.set_xlabel("Date")
ax.set_ylabel("combained_dH")
ax.set_title("Time Series Comparison")
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Compare the correlation of the original and the lowpass filtered dH with TsC

fig, ax = plt.subplots(figsize=(5,5))
ax.scatter(TsC, dH, color="red", s=5)
ax.set_xlabel("TsC_p13")
ax.set_ylabel("dH_p13")
ax.set_title("TsC_dH_p13")

fig, ax = plt.subplots(figsize=(5,5))
ax.scatter(TsC, combined_filtered_signal_low, color="red", s=5)
ax.set_xlabel("TsC_p13")
ax.set_ylabel("filtered_dH_p13")
ax.set_title("lowpass_filtered_TsC_dH_p13")

plt.tight_layout()
plt.show()

### Ondelette + filtre coupe-bande

In [ ]:
# Ondelette analysis of TsC and dH

import pywt

wavelet = 'morl'  # Choose "Morlet" as basis function
scales = np.arange(1, 200)

# Continuous Wavelet Transform (CWT)
coeffs_TsC, freqs = pywt.cwt(TsC, scales, wavelet)
coeffs_dH, freqs = pywt.cwt(dH, scales, wavelet)

In [ ]:
# Visualizing CWT coefficient plot

plt.figure(figsize=(12, 4))
plt.imshow(np.abs(coeffs_TsC), extent=[0, 1, 1, 200], aspect='auto', cmap='jet')
plt.colorbar(label='Magnitude')
plt.title('Continuous Wavelet Transform_TsC')
plt.xlabel('Time')
plt.ylabel('Scale')

plt.figure(figsize=(12, 4))
plt.imshow(np.abs(coeffs_dH), extent=[0, 1, 1, 200], aspect='auto', cmap='jet')
plt.colorbar(label='Magnitude')
plt.title('Continuous Wavelet Transform_dH')
plt.xlabel('Time')
plt.ylabel('Scale')

plt.show()

In [ ]:
# Using a bandstop filter to remove the frequency of the dominant influence of TsC on dH 

from scipy.signal import butter, lfilter

fs = 1/(15*60) # one data every 15min

def butter_bandstop_filter(data, lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='bandstop')
    y = lfilter(b, a, data)
    return y

lowcut = 1/26/3600
highcut = 1/22/3600

In [ ]:
forward_filtered_signal_bandstop = butter_bandstop_filter(data_to_filter, lowcut, highcut, fs, order=4)
reversed_signal = data_to_filter[::-1]
backward_filtered_signal = butter_bandstop_filter(reversed_signal, lowcut, highcut, fs, order=4)
backward_filtered_signal_bandstop = backward_filtered_signal[::-1]


combined_filtered_signal_bandstop = np.zeros(L)

for i in range(L):
    combined_filtered_signal_bandstop[i] =  forward_filtered_signal_bandstop[i] * weight[i] + backward_filtered_signal_bandstop[i] * (1-weight[i])

combined_filtered_dH_bandstop_p13 = pd.DataFrame({'combined_filtered_dH': combined_filtered_signal_bandstop, 'dates': Dates})


fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(Dates, dH, color="red", label="dH_original_p13", s=5)
ax.plot(combined_filtered_dH_bandstop_p13["dates"], combined_filtered_dH_bandstop_p13["combined_filtered_dH"], color="blue", label="combined_filtered_dH_p13")
ax.set_xlabel("Date")
ax.set_ylabel("combained_dH")
ax.set_title("Time Series Comparison")
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Compare the correlation of the original, the lowpass filtered and the bandstop filtered dH with TsC

fig, ax = plt.subplots(figsize=(5,5))
ax.scatter(TsC, dH, color="red", s=5)
ax.set_xlabel("TsC_p13")
ax.set_ylabel("dH_p13")
ax.set_title("TsC_dH_p13")

fig, ax = plt.subplots(figsize=(5,5))
ax.scatter(TsC, combined_filtered_signal_low, color="red", s=5)
ax.set_xlabel("TsC_p13")
ax.set_ylabel("filtered_dH_p13")
ax.set_title("lowpass_filtered_TsC_dH_p13")

fig, ax = plt.subplots(figsize=(5,5))
ax.scatter(TsC, combined_filtered_signal_bandstop, color="red", s=5)
ax.set_xlabel("TsC_p13")
ax.set_ylabel("filtered_dH_p13")
ax.set_title("bandstop_filtered_TsC_dH_p13")

plt.tight_layout()
plt.show()